In [457]:
"""
Python 3.8
"""

# import tensorflow as tf
import tensorflow.compat.v1 as tf
import pandas as pd
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm

'''
do this with user, product, quantity
'''

'\ndo this with user, product, quantity\n'

In [458]:
all_files = ['orders_export_1.csv', 'orders_export_2.csv', 'orders_export_3.csv']
df_from_each_file = (pd.read_csv(f) for f in all_files)
all_orders_original = pd.concat(df_from_each_file, ignore_index=True)

C:\ProgramData\Anaconda3\envs\ALS_DB\lib\site-packages\pandas\core\reshape\concat.py:348: DtypeWarning: Columns (52) have mixed types.Specify dtype option on import or set low_memory=False.
  objs = list(objs)
C:\ProgramData\Anaconda3\envs\ALS_DB\lib\site-packages\pandas\core\reshape\concat.py:348: DtypeWarning: Columns (64) have mixed types.Specify dtype option on import or set low_memory=False.
  objs = list(objs)


In [459]:
# the default dataset
all_orders_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54772 entries, 0 to 54771
Data columns (total 73 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Name                         54772 non-null  object 
 1   Email                        54728 non-null  object 
 2   Financial Status             37480 non-null  object 
 3   Paid at                      30519 non-null  object 
 4   Fulfillment Status           37480 non-null  object 
 5   Fulfilled at                 35612 non-null  object 
 6   Accepts Marketing            37480 non-null  object 
 7   Currency                     37480 non-null  object 
 8   Subtotal                     37480 non-null  float64
 9   Shipping                     37480 non-null  float64
 10  Taxes                        37480 non-null  float64
 11  Total                        37480 non-null  float64
 12  Discount Code                6202 non-null   object 
 13  Discount Amount 

In [460]:
columns = ['Email', 'Lineitem sku', 'Lineitem quantity']


In [461]:
all_orders = all_orders_original[columns]
all_orders.rename(columns={'Lineitem quantity':'Quantity'}, inplace=True)


C:\ProgramData\Anaconda3\envs\ALS_DB\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [462]:
all_orders.head(100)

,Email,Lineitem sku,Quantity
0,natsumi830@hotmail.com,ROUTEINS24,1
1,natsumi830@hotmail.com,Fashion - 31082 - FILA - FILA x BTS GO BEYOND ...,1
2,natsumi830@hotmail.com,Fashion - 31095 - FILA - FILA x BTS GO BEYOND ...,1
3,micahjumaq@gmail.com,Album - 70078 - BTS - 방탄소년단 화양연화 Pt.1 미니3집 - 랜덤,1
4,micahjumaq@gmail.com,"Album - 70077 - BTS - 방탄소년단 O!RUL8,2? 미니1집",1
...,...,...,...
95,huertaareli2001@gmail.com,Fashion - 31584 - FILA - <Project 7> 인터레이션 라이트...,1
96,huertaareli2001@gmail.com,ROUTEINS18,1
97,allinicho10@gmail.com,DBB_Seasonal,1
98,allinicho10@gmail.com,ROUTEINS10,1


In [463]:
all_orders = all_orders.dropna()

In [464]:
all_orders['Email_id'] = all_orders['Email'].astype("category").cat.codes
all_orders['Lineitem sku_id'] = all_orders['Lineitem sku'].astype("category").cat.codes
all_orders.head()

,Email,Lineitem sku,Quantity,Email_id,Lineitem sku_id
0,natsumi830@hotmail.com,ROUTEINS24,1,10157,4304
1,natsumi830@hotmail.com,Fashion - 31082 - FILA - FILA x BTS GO BEYOND ...,1,10157,2536
2,natsumi830@hotmail.com,Fashion - 31095 - FILA - FILA x BTS GO BEYOND ...,1,10157,2543
3,micahjumaq@gmail.com,Album - 70078 - BTS - 방탄소년단 화양연화 Pt.1 미니3집 - 랜덤,1,9488,78
4,micahjumaq@gmail.com,"Album - 70077 - BTS - 방탄소년단 O!RUL8,2? 미니1집",1,9488,77


In [465]:
user_lookup = all_orders[['Email_id', 'Email']].drop_duplicates()
user_lookup['Email_id'] = user_lookup['Email_id'].astype(str)

In [466]:
item_lookup = all_orders[['Lineitem sku_id', 'Lineitem sku']].drop_duplicates()
item_lookup['Lineitem sku_id'] = item_lookup['Lineitem sku_id'].astype(str)
item_lookup

,Lineitem sku_id,Lineitem sku
0,4304,ROUTEINS24
1,2536,Fashion - 31082 - FILA - FILA x BTS GO BEYOND ...
2,2543,Fashion - 31095 - FILA - FILA x BTS GO BEYOND ...
3,78,Album - 70078 - BTS - 방탄소년단 화양연화 Pt.1 미니3집 - 랜덤
4,77,"Album - 70077 - BTS - 방탄소년단 O!RUL8,2? 미니1집"
...,...,...
53214,14,ADD-ON ETUDEHOUSE_AC_CLEANSER
53858,22,ADD5
53874,19,ADD2
53926,18,ADD1


In [467]:
all_orders = all_orders.drop(['Email', 'Lineitem sku'], axis=1)

In [468]:
all_orders = all_orders.loc[all_orders['Quantity'] != 0]
all_orders.head()

,Quantity,Email_id,Lineitem sku_id
0,1,10157,4304
1,1,10157,2536
2,1,10157,2543
3,1,9488,78
4,1,9488,77


In [469]:
users = list(np.sort(all_orders['Email_id'].unique()))
users

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [470]:
items = list(np.sort(all_orders['Lineitem sku_id'].unique()))
items

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [471]:
quantity = list(all_orders['Quantity'])
quantity

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [472]:
# Get the rows and columns for our new matrix
rows = all_orders['Email_id'].astype(float)
cols = all_orders['Lineitem sku_id'].astype(float)

# rows = np.array(all_orders['Email_id'], dtype=np.float)
# rows = np.array(all_orders['Lineitem sku_id'], dtype=np.float)

# Contruct a sparse matrix for our users and items containing number of plays
data_sparse = sp.csr_matrix((quantity, (rows, cols)), shape=(len(users), len(items)))
data_sparse

<14144x4868 sparse matrix of type '<class 'numpy.intc'>'
	with 37201 stored elements in Compressed Sparse Row format>

In [473]:
# Get the values of our matrix as a list of user ids and item ids.
uids, iids = data_sparse.nonzero()

In [474]:
"""
HYPERPARAMS
"""

epochs = 50
batches = 30
num_factors = 32 # Number of latent features

# Independent lambda regularization values or user, items and bias.
lambda_user = 0.0000001
lambda_item = 0.0000001
lambda_bias = 0.0000001

# Our learning rate
lr = 0.005

# How many (u,i,j) triplets we sample for each batch
samples = 100

"""
TENSORFLOW GRAPH
"""

# Set up our Tensorflow graph
graph = tf.Graph()

def init_variable(size, dim, name=None):
    '''
    Helper function to initialize a new variable with
    uniform random values.
    '''
    std = np.sqrt(2 / dim)
    return tf.Variable(tf.random_uniform([size, dim], -std, std), name=name)


def embed(inputs, size, dim, name=None):
    '''
    Helper function to get a Tensorflow variable and create
    an embedding lookup to map our user and item
    indices to vectors.
    '''
    emb = init_variable(size, dim, name)
    return tf.nn.embedding_lookup(emb, inputs)


def get_variable(graph, session, name):
    '''
    Helper function to get the value of a
    Tensorflow variable by name.
    '''
    v = graph.get_operation_by_name(name)
    v = v.values()[0]
    v = v.eval(session=session)
    return v

with graph.as_default():
    '''
    Loss function:
    -SUM ln σ(xui - xuj) + λ(w1)**2 + λ(w2)**2 + λ(w3)**2 ...
    ln = the natural log
    σ(xuij) = the sigmoid function of xuij.
    λ = lambda regularization value.
    ||W||**2 = the squared L2 norm of our model parameters.

    '''

    # Input into our model, in this case our user (u),
    # known item (i) an unknown item (i) triplets.
    u = tf.placeholder(tf.int64, shape=(None, 1))
    i = tf.placeholder(tf.int64, shape=(None, 1))
    j = tf.placeholder(tf.int64, shape=(None, 1))

    # User feature embedding
    u_factors = embed(u, len(users), num_factors, 'user_factors') # U matrix

    # Known and unknown item embeddings
    item_factors = init_variable(len(items), num_factors, "item_factors") # V matrix
    i_factors = tf.nn.embedding_lookup(item_factors, i)
    j_factors = tf.nn.embedding_lookup(item_factors, j)

    # i and j bias embeddings.
    item_bias = init_variable(len(items), 1, "item_bias")
    i_bias = tf.nn.embedding_lookup(item_bias, i)
    i_bias = tf.reshape(i_bias, [-1, 1])
    j_bias = tf.nn.embedding_lookup(item_bias, j)
    j_bias = tf.reshape(j_bias, [-1, 1])

    # Calculate the dot product + bias for known and unknown item to get xui and xuj.
    xui = i_bias + tf.reduce_sum(u_factors * i_factors, axis=2)
    xuj = j_bias + tf.reduce_sum(u_factors * j_factors, axis=2)

    # We calculate xuij.
    xuij = xui - xuj

    # Calculate the mean AUC (area under curve). if xuij is greater than 0, that means that
    # xui is greater than xuj (and thats what we want).
    u_auc = tf.reduce_mean(tf.to_float(xuij > 0))

    # Output the AUC value to tensorboard for monitoring.
    tf.summary.scalar('auc', u_auc)

    # Calculate the squared L2 norm ||W||**2 multiplied by λ.
    l2_norm = tf.add_n([
        lambda_user * tf.reduce_sum(tf.multiply(u_factors, u_factors)),
        lambda_item * tf.reduce_sum(tf.multiply(i_factors, i_factors)),
        lambda_item * tf.reduce_sum(tf.multiply(j_factors, j_factors)),
        lambda_bias * tf.reduce_sum(tf.multiply(i_bias, i_bias)),
        lambda_bias * tf.reduce_sum(tf.multiply(j_bias, j_bias))
        ])

    # Calculate the loss as ||W||**2 - ln σ(Xuij)
    # loss = l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(xuij)))
    loss = -tf.reduce_mean(tf.log(tf.sigmoid(xuij))) + l2_norm

    # Train using the Adam optimizer to minimize our loss function.
    opt = tf.train.AdamOptimizer(learning_rate=lr)
    step = opt.minimize(loss)

    # Initialize all tensorflow variables.
    init = tf.global_variables_initializer()

In [475]:
len(items)

4868

In [476]:
len(users)

14144

In [477]:
"""
GRAPH EXECUTION
"""

# Run the session.
session = tf.Session(config=None, graph=graph)
session.run(init)

progress = tqdm(total=batches*epochs)

for _ in range(epochs):
    for _ in range(batches):

        # We want to sample one known and one unknown item for each user sample
        idx = np.random.randint(low=0, high=len(uids), size=samples)

        # We then grab the users matching those indices
        batch_u = uids[idx].reshape(-1, 1)

        # Then the known items for those users
        batch_i = iids[idx].reshape(-1, 1)

        # Lastly we randomly sample one unknown item for each user
        batch_j = np.random.randint(low=0, high=len(items), size=(samples, 1), dtype=np.int64)

        # Feed our users, known and unknown items to our tensorflow graph
        feed_dict = { u: batch_u, i: batch_i, j: batch_j }

        # We run the session
        _, l, auc = session.run([step, loss, u_auc], feed_dict)

    progress.update(batches)
    progress.set_description('Loss: %.3f | AUC: %.3f' % (l, auc))

progress.close()

Loss: 0.102 | AUC: 0.980: 100%|██████████| 1500/1500 [00:05<00:00, 254.31it/s]


In [ ]:
def find_similar_items(item=None, num_items=5):
    """Find items similar to an item
    Args:
        item (str): The name of the item we want to find similar items for
        num_items (int): How many similar items we want to return.
    Returns:
        similar (pandas.DataFrame): DataFrame with num_items item names and scores
    """

    # Grab our User matrix U
    user_vecs = get_variable(graph, session, 'user_factors')

    # Grab our Item matrix V
    item_vecs = get_variable(graph, session, 'item_factors')

    # Grab our item bias
    item_bi = get_variable(graph, session, 'item_bias').reshape(-1)

    # Get the item id
    item_id = int(item_lookup[item_lookup['Lineitem sku'] == item]['Lineitem sku_id'])

    # Get the item vector for our item_id and transpose it.
    item_vec = item_vecs[item_id].T

    # Calculate the similarity between by multiplying the item vector with our item_matrix
    scores = np.add(item_vecs.dot(item_vec), item_bi).reshape(1,-1)[0]

    # Get the indices for the top 5 scores
    top_10 = np.argsort(scores)[::-1][:num_items]

    # We then use our lookup table to grab the names of these indices
# and add it along with its score to a pandas dataframe.
    items, item_scores = [], []

    for idx in top_5:
        items.append(item_lookup['Lineitem sku'].loc[item_lookup['Lineitem sku_id'] == str(idx)].iloc[0])
        item_scores.append(scores[idx])

    similar = pd.DataFrame({'Lineitem sku': items, 'score': item_scores})
    return similar

print(find_similar_items(item=' '))



'''
TEST
'''

In [ ]:
item='Album - 70078 - BTS - 방탄소년단 화양연화 Pt.1 미니3집 - 랜덤'
num_items=5

In [ ]:
user_vecs = get_variable(graph, session, 'user_factors')
user_vecs

In [ ]:
item_vecs = get_variable(graph, session, 'item_factors')
item_vecs

In [ ]:
item_bi = get_variable(graph, session, 'item_bias').reshape(-1)
item_bi

In [ ]:
item_id = int(item_lookup[item_lookup['Lineitem sku'] == item]['Lineitem sku_id'])
item_id

In [ ]:
item_vec = item_vecs[item_id].T

In [ ]:
item_vec

In [ ]:
scores = np.add(item_vecs.dot(item_vec), item_bi).reshape(1,-1)[0]

In [ ]:
top_5 = np.argsort(scores)[::-1][:num_items]

In [ ]:
items, item_scores = [], []

In [ ]:
for idx in top_5:
    items.append(item_lookup['Lineitem sku'].loc[item_lookup['Lineitem sku_id'] == str(idx)].iloc[0])
    item_scores.append(scores[idx])

    similar = pd.DataFrame({'Lineitem sku': items, 'score': item_scores})
    print(similar)